In [1]:
! make

/usr/local/cuda-8.0/bin/nvcc -o master.exe main.cu -std=c++11 -I"/use/local/cuda/samples/common/inc" -lGL -lGLU -lglut
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
main.cu(658): warning: variable "dim" was declared but never referenced

main.cu(658): warning: variable "n" was declared but never referenced

main.cu(897): warning: variable "this_p" was set but never used

main.cu(897): warning: variable "this_v" was set but never used

main.cu(658): warning: variable "dim" was declared but never referenced

main.cu(658): warning: variable "n" was declared but never referenced

main.cu(897): warning: variable "this_p" was set but never used

main.cu(897): warning: variable "this_v" was set but never used

main.cu: In function 'void read_input_file_and_set_initial_conditions()':
main.cu:792:52: warning: format '%d' expects argument of type 'int*', but argumen

In [1]:
# %load setup.py
import itertools as it
import time
import datetime
import os
import numpy as np
import pandas as pd
idx = pd.IndexSlice

import IPython.display as ipd
digits = 3
pd.options.display.chop_threshold = 10**-(digits+1)
#pd.options.display.precision = digits
pd.options.display.float_format = lambda x: '{0:.{1}f}'.format(x,digits)
pd.options.display.show_dimensions = True
def display(X):
    if isinstance(X, np.ndarray) or isinstance(X, pd.Series):
        ipd.display(pd.DataFrame(X))
    else:
        ipd.display(X)

import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook
plt.style.use("classic")
plt.style.use("fivethirtyeight")
#plt.style.use("bmh")
plt.rc("figure", figsize=(5,3))

def Lnorm(v,p=2):
    if p%2 == 0:
        return (v**p).sum()**(1/p)
    else:
        return (v.abs()**p).sum()**(1/p)
    
def Ldist(v,w,p=2):
    return Lnorm(v-w,p)

In [2]:
default_params = { 'dimension' : 2
                  ,'num_particles' : 2
                  ,'num_walls' : 4
                  ,'mass' : 2.0
                  ,'radius' : 1.0
                  ,'gamma' : 0.7071
                  ,'offset' : 0
                  ,'spin' : 0.0
                  ,'num_steps' : 10000
                  ,'done' : False
                  ,'run_time' : 0.0
                  ,'folder' : ''
                 }

def listify(y):
    if isinstance(y,str):
        return [y]
    elif hasattr(y,'__iter__'):
        return y
    else:
        return [y]

In [28]:
params = default_params.copy()

#set ranges for parameters to vary
params['offset'] = np.linspace(start=0.0, stop=0.05, num=3)
params['radius'] = np.arange(start=0.3, stop=0.5+.1, step=.1,)
#params['mass'] = np.linspace(10,15,7)

params.update((x, listify(y)) for x, y in params.items())
experiments = pd.DataFrame(list(it.product(*params.values())),
                           columns=params.keys())

def run_many_experiments(experiments):
    start_run = time.time()
    global base_directory, run_folder
    base_directory = os.getcwd()
    while True:
        run_folder = base_directory +'/'+time.strftime("%Y:%m:%d--%H:%M:%S")
        if not os.path.exists(run_folder):
            os.makedirs(run_folder)
            break
    run_file = run_folder + '/experiment_list.csv'
    #with open(run_folder + '/experiment_list.csv', 'w') as run_file:
    #experiments = experiments.apply(run_single_experiment,axis=1)
    #The above line would be nicer, but apply runs the first row twice.  The second time overwrites the first time.
    #Thus, we need to run the entire first experiment twice and throw away one.  This is inefficient.
    pd.options.mode.chained_assignment = None
    for i in range(experiments.shape[0]):        
        experiments.ix[i] = run_single_experiment(experiments.ix[i])
        experiments.to_csv(run_file, mode='w')
    elapsed = time.time() - start_run
    elapsed = str(datetime.timedelta(seconds=round(elapsed,0)))
    print('\nRun complete - I did %d experiments.  It took %s'%(i,elapsed))
    with open(run_file, 'a') as run_file:
        run_file.write('TIME ELAPSED = %s'%elapsed)
    return experiments
            

def run_single_experiment(exp):
    start_exp = time.time()
    exp_folder = run_folder + '/' + str(exp.name).zfill(3)
    if os.path.exists(exp_folder):
        os.rmdir(exp_folder)
    os.makedirs(exp_folder)
    exp['folder'] = exp_folder
    if exp.name < 1:
        exp['num_steps'] = 10000

    Walls = [[4, 0, 0, -100000.0,  1.0, 0.0,  100000.0,  1.0, 0.0],
             [5, 0, 0,  100000.0,  1.0, 0.0,  100000.0, -1.0, 0.0],
             [4, 0, 0,  100000.0, -1.0, 0.0, -100000.0, -1.0, 0.0],
             [5, 0, 0, -100000.0, -1.0, 0.0, -100000.0,  1.0, 0.0]]

    pos1 = np.array([exp['offset'], 1 - 1.1*exp['radius'], 0.0])
    pos2 = np.array([-exp['offset'], -1 + 1.1*exp['radius'], 0.0])
    #pos1 = np.array([exp['offset'], 0.5, 0.0])
    #pos2 = np.array([-exp['offset'], -0.5, 0.0])
    
    velocitydir = pos1 - pos2
    velocitydir = velocitydir / np.linalg.norm(velocitydir)

    vel1 =-0.5 * velocitydir
    vel2 = 0.5 * velocitydir
    
    #p1=spin+g+' '.join([str(p)for p in pos1])+R+mass+' '.join([str(np.round(v, 4))for v in vel1])
    #p2=spin+g+' '.join([str(p)for p in pos2])+R+mass+' '.join([str(np.round(v, 4))for v in vel2])
    p1 = [exp['spin'], exp['gamma'], *pos1, exp['radius'], exp['mass'], *vel1]    
    p1 = ' '.join(map(str, p1))
    p2 = [exp['spin'], exp['gamma'], *pos2, exp['radius'], exp['mass'], *vel2]    
    p2 = ' '.join(map(str, p2))
    
    with open(exp_folder + '/no_slip_input_file', 'w') as parameter_file:
        parameter_file.write('# dimension\n%s\n'% exp['dimension'])
        parameter_file.write('# number of gas particles\n%s\n'%exp['num_particles'])
        parameter_file.write('# size of gas particles\n%s\n'%exp['radius'])
        parameter_file.write('# number of collision steps\n%s\n'%exp['num_steps'])
        parameter_file.write('# number of walls\n%s\n'%exp['num_walls'])
        parameter_file.write('# wall type and temp and endpoints\n')
        for wall in Walls: 
            newline = ' '.join([str(a) for a in wall])+'\n'
            parameter_file.write(newline)

        parameter_file.write('# no_slip particles? \n1 \n')
        parameter_file.write('#initialize particles? a,g,px,py,pz,r,m,vx,vy,vz\n1\n')
        parameter_file.write(p1+'\n')
        parameter_file.write(p2+'\n')
        parameter_file.write('#output directory\n')
        parameter_file.write(exp_folder+'/no_slip_\n')        
    
    os.system(base_directory + "/master.exe " + exp_folder +"/no_slip_input_file")
    exp['done'] = True
    elapsed = time.time()-start_exp
    elapsed = str(datetime.timedelta(seconds=round(elapsed,0)))
    exp['run_time'] = elapsed
    print('\nExperiment %d complete.  Elapsed time = %s'%(exp.name,elapsed))
    return exp

#display(experiments)
experiments = run_many_experiments(experiments)
display(experiments)


Experiment 0 complete.  Elapsed time = 0:00:02

Experiment 1 complete.  Elapsed time = 0:00:01

Experiment 2 complete.  Elapsed time = 0:00:01

Experiment 3 complete.  Elapsed time = 0:00:01

Experiment 4 complete.  Elapsed time = 0:00:01

Experiment 5 complete.  Elapsed time = 0:00:01

Experiment 6 complete.  Elapsed time = 0:00:01

Experiment 7 complete.  Elapsed time = 0:00:01

Experiment 8 complete.  Elapsed time = 0:00:01

Run complete - I did 8 experiments.  It took 0:00:13


,dimension,num_particles,num_walls,mass,radius,gamma,offset,spin,num_steps,done,run_time,folder
0,2,2,4,2.000,0.300,0.707,0.000,0.000,10000,True,0:00:02,/media/storage/Billiards/dev/cooksandbox/2017:...
1,2,2,4,2.000,0.300,0.707,0.025,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
2,2,2,4,2.000,0.300,0.707,0.050,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
3,2,2,4,2.000,0.400,0.707,0.000,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
4,2,2,4,2.000,0.400,0.707,0.025,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
5,2,2,4,2.000,0.400,0.707,0.050,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
6,2,2,4,2.000,0.500,0.707,0.000,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
7,2,2,4,2.000,0.500,0.707,0.025,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
8,2,2,4,2.000,0.500,0.707,0.050,0.000,10000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...


In [3]:
base_directory = os.getcwd()
run_folder = base_directory + '/2017:03:06--21:23:30'
run_file = run_folder + '/experiment_list.csv'
experiments_read = pd.read_csv(run_file,index_col=0)#,skipfooter=1)
keep = ~(experiments_read.isnull().all(axis=1))
experiments_read = experiments_read.ix[keep]
experiments_read.index = experiments_read.index.astype(int)
display(experiments_read)
display(experiments_read.index)

,dimension,num_particles,num_walls,mass,radius,gamma,offset,spin,num_steps,done,run_time,folder
0,2.000,2.000,4.000,2.000,0.300,0.707,0.000,0.000,10000.000,True,0:00:02,/media/storage/Billiards/dev/cooksandbox/2017:...
1,2.000,2.000,4.000,2.000,0.300,0.707,0.025,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
2,2.000,2.000,4.000,2.000,0.300,0.707,0.050,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
3,2.000,2.000,4.000,2.000,0.400,0.707,0.000,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
4,2.000,2.000,4.000,2.000,0.400,0.707,0.025,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
5,2.000,2.000,4.000,2.000,0.400,0.707,0.050,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
6,2.000,2.000,4.000,2.000,0.500,0.707,0.000,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
7,2.000,2.000,4.000,2.000,0.500,0.707,0.025,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...
8,2.000,2.000,4.000,2.000,0.500,0.707,0.050,0.000,10000.000,True,0:00:01,/media/storage/Billiards/dev/cooksandbox/2017:...


Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [81]:
#display(experiments_read)
display(np.isreal(experiments_read.index[-3]))
experiments_read.applymap(np.isreal)
y = experiments_read.index.tolist()
z = [isinstance(w,int) for w in y]
z
keep = ~(experiments_read.isnull().all(axis=1))

/media/storage/Billiards/dev/cooksandbox/2017:03:06--21:23:30
/media/storage/Billiards/dev/cooksandbox/2017:03:06--21:23:30/experiment_list.csv
000/  001/  002/  003/  004/  005/  006/  007/  008/  experiment_list.csv


False

0                          True
1                          True
2                          True
3                          True
4                          True
5                          True
6                          True
7                          True
8                          True
TIME ELAPSED = 0:00:13    False
dtype: bool

In [34]:
def run_experiment(): 
    global base_directory, run_folder

    run_folder = base_directory +'/'+time.strftime("%Y:%m:%d--%H:%M:%S")
    if not os.path.exists(run_folder):
        positions  = [0.5, -0.5]
        NWALLS = '4'
        N = '2'
        D = '2'
        R = ' 0.4 '
        mass = ' 2.0 '
        spin = ' 0.0 '
        g = ' 0.7071 '
        offsets = np.linspace(0.0, 0.05, num=21)

        experimentcounter = 0
        
        for val in offsets: 
        folder = run_folder + '/' + str(i).zfill(3)
        os.makedirs(folder)
        os.chdir(str(folder))
        Walls = [[4, 0, 0, -100000.0,  1.0, 0.0,  100000.0,  1.0, 0.0], \
                 [5, 0, 0,  100000.0,  1.0, 0.0,  100000.0, -1.0, 0.0], \
                 [4, 0, 0,  100000.0, -1.0, 0.0, -100000.0, -1.0, 0.0], \
                 [5, 0, 0, -100000.0, -1.0, 0.0, -100000.0,  1.0, 0.0]]
    
        if i < 1: 
            exp['num_steps'] = 10000

    experimentcounter = 0
    for val in offsets: 
        #this_experiment = run_folder + '/' + str(experimentcounter).zfill(3)
        #os.makedirs(this_experiment)
        #os.chdir(str(this_experiment))

        #Walls = [[4, 0, 0, -100000.0,  1.0, 0.0,  100000.0,  1.0, 0.0], \
                 [5, 0, 0,  100000.0,  1.0, 0.0,  100000.0, -1.0, 0.0], \
                 [4, 0, 0,  100000.0, -1.0, 0.0, -100000.0, -1.0, 0.0], \
                 [5, 0, 0, -100000.0, -1.0, 0.0, -100000.0,  1.0, 0.0]]

        #NSTEPS = '1000000'
        if experimentcounter < 1: 
            NSTEPS = '10000'
        pos1 = np.asarray([val, positions[0], 0.0])
        pos2 = np.asarray([-val, positions[1], 0.0])

        velocitydir = np.asarray((pos1 - pos2))
        velocitydir = velocitydir / np.linalg.norm(velocitydir)

        vel1 =-0.5 * velocitydir
        vel2 = 0.5 * velocitydir

        p1=spin+g+' '.join([str(p)for p in pos1])+R+mass+' '.join([str(np.round(v, 4))for v in vel1])
        p2=spin+g+' '.join([str(p)for p in pos2])+R+mass+' '.join([str(np.round(v, 4))for v in vel2])

        parameter_file = open('no_slip_input_file','w')
        parameter_file.write('# dimension\n%s\n'%D)
        parameter_file.write('# number of gas particles\n%s\n'%N)
        parameter_file.write('# size of gas particles\n%s\n'%R)
        parameter_file.write('# number of collision steps\n%s\n'%NSTEPS)
        parameter_file.write('# number of walls\n%s\n'%NWALLS)
        parameter_file.write('# wall type and temp and endpoints\n')
        for wall in Walls: 
            newline = ' '.join([str(a) for a in wall])+'\n'
            parameter_file.write(newline)

        parameter_file.write('# no_slip particles? \n1 \n')
        parameter_file.write('#initialize particles? a,g,px,py,pz,r,m,vx,vy,vz\n1\n')
        parameter_file.write(p1+'\n')
        parameter_file.write(p2+'\n')
        parameter_file.write('#output directory\n')
        parameter_file.write(str(this_experiment)+'/no_slip_\n')
        parameter_file.close()
        print('\nIteration%d'%experimentcounter)
        os.system(base_directory + "/master.exe no_slip_input_file")

        Walls = [[0, 0, 0, -100000.0,  1.0, 0.0,  100000.0,  1.0, 0.0], \
                 [5, 0, 0,  100000.0,  1.0, 0.0,  100000.0, -1.0, 0.0], \
                 [0, 0, 0,  100000.0, -1.0, 0.0, -100000.0, -1.0, 0.0], \
                 [5, 0, 0, -100000.0, -1.0, 0.0, -100000.0,  1.0, 0.0]]

        parameter_file = open('slip_input_file','w')
        parameter_file.write('# dimension\n%s\n'%D)
        parameter_file.write('# number of gas particles\n%s\n'%N)
        parameter_file.write('# size of gas particles\n%s\n'%R)
        parameter_file.write('# number of collision steps\n%s\n'%NSTEPS)
        parameter_file.write('# number of walls\n%s\n'%NWALLS)
        parameter_file.write('# wall type and temp and endpoints\n')
        for wall in Walls: 
            newline = ' '.join([str(a) for a in wall])+'\n'
            parameter_file.write(newline)
        parameter_file.write('# no_slip particles? \n0 \n')
        parameter_file.write('#initialize particles? a,g,px,py,pz,r,m,vx,vy,vz\n1\n')
        parameter_file.write(p1+'\n')
        parameter_file.write(p2+'\n')
        parameter_file.write('#output directory\n')
        parameter_file.write(str(this_experiment)+'/slip_\n')
        parameter_file.close()
        print('\nIteration%d'%experimentcounter)
        os.system(base_directory + "/master.exe slip_input_file")
        experimentcounter += 1
    return exp

'/media/storage/Billiards/dev/cooksandbox'

AttributeError: module 'os' has no attribute 'getpwd'

In [25]:
stud = 30*(100+5+20)
veh = 4*(80*3)
print(stud,veh,stud+veh)

3750 960 4710
